In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EcommerceWorkspace") \
    .enableHiveSupport() \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 17:14:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.setLogLevel("ERROR")

In [4]:
SparkSession.builder.getOrCreate()

In [5]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS ecommerce
""")

DataFrame[]

In [6]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|ecommerce|
+---------+



In [7]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS ecommerce_data
""")

DataFrame[]

In [8]:
import os

os.environ["KAGGLE_USERNAME"] = "ernidhi"
os.environ["KAGGLE_KEY"] = "KGAT_1c8e2ae620ede4cb27a4f19a633c0a60"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [9]:
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
100%|██████████████████████████████████████▉| 4.28G/4.29G [00:04<00:00, 456MB/s]
100%|██████████████████████████████████████| 4.29G/4.29G [00:04<00:00, 1.14GB/s]


In [10]:
!unzip ecommerce-behavior-data-from-multi-category-store.zip -d ecommerce_data

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: ecommerce_data/2019-Nov.csv  
  inflating: ecommerce_data/2019-Oct.csv  


In [11]:
!cd ecommerce_data && ls -lh

total 14G
-rw-r--r-- 1 root root 8.4G Dec  9  2019 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Dec  9  2019 2019-Oct.csv


In [12]:
!rm -f ecommerce-behavior-data-from-multi-category-store.zip

In [13]:
!ls -lh

total 48K
-rw-r--r-- 1 root root  686 Jan 11 17:14 derby.log
drwxr-xr-x 2 root root 4.0K Jan 11 17:18 ecommerce_data
drwxr-xr-x 5 root root 4.0K Jan 11 17:14 metastore_db
---------- 1 root root  29K Jan 11 17:18 __notebook__.ipynb
drwxr-xr-x 4 root root 4.0K Jan 11 17:14 spark-warehouse


In [14]:
!pwd
!ls



/kaggle/working
derby.log  ecommerce_data  metastore_db  __notebook__.ipynb  spark-warehouse


In [15]:
!ls ecommerce_data


2019-Nov.csv  2019-Oct.csv


In [16]:
events = spark.read.csv(
    "ecommerce_data/*.csv",
    header=True,
    inferSchema=True
)


In [17]:
events.show(5)
events.printSchema()
events.count()


+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|  17302664|2053013553853497655|                NULL| creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|    lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.smart...|xiaomi

109950743

In [18]:
events.select("event_type", "product_id", "brand", "price").show(20)


+----------+----------+--------+------+
|event_type|product_id|   brand| price|
+----------+----------+--------+------+
|      view|   1003461|  xiaomi|489.07|
|      view|   5000088|  janome|293.65|
|      view|  17302664|   creed| 28.31|
|      view|   3601530|      lg|712.87|
|      view|   1004775|  xiaomi|183.27|
|      view|   1306894|      hp|360.09|
|      view|   1306421|      hp|514.56|
|      view|  15900065| rondell| 30.86|
|      view|  12708937|michelin| 72.72|
|      view|   1004258|   apple|732.07|
|      view|  17200570|    NULL|437.33|
|      view|   2701517|    NULL|155.11|
|      view|  16700260|    NULL| 31.64|
|      view|  34600011|    NULL| 20.54|
|      view|   4600658| samsung|411.83|
|      view|  24900193|    NULL|  1.09|
|      view|  27400066|    NULL|  8.55|
|      view|   5100503|  xiaomi| 22.68|
|      view|   1004566|  huawei|164.84|
|      view|   1307115|      hp|411.59|
+----------+----------+--------+------+
only showing top 20 rows



In [19]:
events.filter(events.price > 1500).show(5)


+-------------------+----------+----------+-------------------+------------------+------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|     category_code| brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+------------------+------+-------+---------+--------------------+
|2019-11-01 00:00:51|      view|   1307012|2053013558920217191|computers.notebook| apple|2342.15|530857208|c69f9c63-7098-426...|
|2019-11-01 00:01:03|      view|   1306571|2053013558920217191|computers.notebook|  acer| 2312.8|530857208|c69f9c63-7098-426...|
|2019-11-01 00:01:05|      view|   5100797|2053013553341792533|electronics.clocks|garmin|2239.44|521368162|375da15e-023b-47a...|
|2019-11-01 00:01:08|      view|   1305977|2053013558920217191|computers.notebook| apple|2496.59|530857208|c69f9c63-7098-426...|
|2019-11-01 00:01:24|      view|   1304315|2053013558920217191|computers.notebook| apple|2311.49|

In [20]:
events.filter(events.price.cast('double')>2500).select('brand').distinct().orderBy('brand').show(5)

+-----+
|brand|
+-----+
| NULL|
| acer|
|alser|
|apple|
| asus|
+-----+
only showing top 5 rows



In [21]:
events.filter((events.event_type == "view") & (events.price.cast("double")>2500)).groupBy("brand").count().orderBy("brand").show(10)

+-------+-----+
|  brand|count|
+-------+-----+
|   NULL|26521|
|   acer|21213|
|  alser|   86|
|  apple|19333|
|   asus| 7729|
|bernina|  382|
|brother|    5|
|buderus|  251|
|  canon| 5353|
|cartier| 3165|
+-------+-----+
only showing top 10 rows



In [22]:
events.filter((events.event_type == "purchase") & (events.price.cast("double")>2500)).groupBy("brand").count().orderBy("brand").show(10)

+-------+-----+
|  brand|count|
+-------+-----+
|   NULL|   57|
|   acer|   26|
|  apple|  105|
|   asus|   19|
|bernina|    1|
|  canon|   43|
|cartier|    2|
| garmin|    2|
|  giant|   12|
|   jura|    2|
+-------+-----+
only showing top 10 rows

